In [ ]:
# default_exp preprocessing

# Preprocessing

> Este modulo es el encargado de la carga, preprocesamiento y preparación de los daros.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def sum(a,b):
    "Suma dos números"
    return a + b

In [ ]:
assert sum(4,3)==7